#### Testing Local Container (without SM SDK)

In [ ]:
from matplotlib import pyplot as plt

import cv2
import requests

from PIL import Image
import json
import base64
from io import BytesIO

In [ ]:
im = cv2.imread("./desk.jpg")
plt.figure(figsize = (60,20))
plt.imshow(im)
plt.show()

### Run Local Container
At this point, local detic container must be running.
If there's no container running (could be checked with `docker ps` command), just start a new one:

```
docker run -v /opt/ml/model:/opt/ml/model -p 8080:8080 --rm detic-sage-maker serve
```

In [ ]:
# Testing ping
response = requests.get(url="http://127.0.0.1:8080/ping")
print(response)

In [ ]:
# Prepare Image to send
img = Image.open("desk.jpg")

#Convert Pillow Image to bytes and then to base64
buffered = BytesIO()
img.save(buffered, format="JPEG")
img_byte = buffered.getvalue() # bytes
img_base64 = base64.b64encode(img_byte) #Base64-encoded bytes * not str

#It's still bytes so json.Convert to str to dumps(Because the json element does not support bytes type)
img_str = img_base64.decode('utf-8') # str

files = {
    "labels":"mouse,cup", #change to labels that will be detected
    "img":img_str
}

In [ ]:
# Testing inference on model
response = requests.post(url="http://127.0.0.1:8080/invocations",json=json.dumps(files))

print(response)

In [ ]:
# Detectron2 to load response
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import jsonpickle

metadata = MetadataCatalog.get("__unused")

In [ ]:
frame = jsonpickle.decode(response.text)

v = Visualizer(im[:, :, ::-1], metadata)
out = v.draw_instance_predictions(frame["instances"].to("cpu"))
plt.figure(figsize = (60,20))
plt.imshow(out.get_image())
plt.show()